In [3]:
import tensorflow as tf 

In [4]:
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
  '../datasets/train',
  image_size=(150,150),
  batch_size=64,
  subset='training',
  validation_split=0.2,
  seed=1234
)

val_ds = tf.keras.preprocessing.image_dataset_from_directory(
  '../datasets/',
  image_size=(150,150),
  batch_size=64,
  subset='validation',
  validation_split=0.2,
  seed=1234
)

Found 25000 files belonging to 2 classes.
Using 20000 files for training.
Metal device set to: Apple M1

systemMemory: 8.00 GB
maxCacheSize: 2.67 GB



2023-06-01 12:20:46.553590: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:303] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-06-01 12:20:46.553612: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:269] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Found 37500 files belonging to 2 classes.
Using 7500 files for validation.


In [5]:
print(train_ds)

def 전처리함수(i, 정답):
  i = tf.cast( i/255.0, tf.float32 )
  return i, 정답

train_ds = train_ds.map(전처리함수)
val_ds = val_ds.map(전처리함수)

<_BatchDataset element_spec=(TensorSpec(shape=(None, 150, 150, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int32, name=None))>


In [6]:
# 구글 모델 훔쳐오기
import requests

url = 'https://storage.googleapis.com/mledu-datasets/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5'
r = requests.get(url, allow_redirects=True)

open('inception_v3.h5', 'wb').write(r.content)

87910968

include_top: __**마지막 출력중 Dense레이어**__(Top Layer)를 포함할지 여부

In [10]:
from tensorflow.keras.applications.inception_v3 import InceptionV3

inception_model = InceptionV3( input_shape=(150,150,3), include_top=False, weights=None)
inception_model.load_weights('./inception_v3.h5')

inception_model.summary()

Model: "inception_v3"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_2 (InputLayer)        [(None, 150, 150, 3)]        0         []                            
                                                                                                  
 conv2d_94 (Conv2D)          (None, 74, 74, 32)           864       ['input_2[0][0]']             
                                                                                                  
 batch_normalization_94 (Ba  (None, 74, 74, 32)           96        ['conv2d_94[0][0]']           
 tchNormalization)                                                                                
                                                                                                  
 activation_94 (Activation)  (None, 74, 74, 32)           0         ['batch_normalizati

In [8]:
# 모델을 시각적으로 보기
from tensorflow.keras.utils import plot_model

plot_model(inception_model, to_file='model.png', show_shapes=True, show_layer_names=True)

#### 가중치고정 + Fine Tuning

In [14]:
# Model을 불러온 경우 학습시키면 안되기때문에 학습 금지 설정
for i in inception_model.layers:
  i.trainable = False

In [21]:
# 그런데 일부 레이어는 미세 조정을해서 성능 향상을 가져올 수도 있음
for i in inception_model.layers:
  if i.name == 'mixed6':
    unfreeze = True
    if unfreeze == True:
      i.trainable = True

In [22]:
마지막레이어 = inception_model.get_layer('mixed7')

flatten1 = tf.keras.layers.Flatten()(마지막레이어.output)
dense1 = tf.keras.layers.Dense(1024, activation='relu')(flatten1)
drop1 = tf.keras.layers.Dropout(0.2)(dense1)
output = tf.keras.layers.Dense(1, activation='sigmoid')(drop1)

model = tf.keras.Model(inception_model.input, output)
model.summary()

Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_2 (InputLayer)        [(None, 150, 150, 3)]        0         []                            
                                                                                                  
 conv2d_94 (Conv2D)          (None, 74, 74, 32)           864       ['input_2[0][0]']             
                                                                                                  
 batch_normalization_94 (Ba  (None, 74, 74, 32)           96        ['conv2d_94[0][0]']           
 tchNormalization)                                                                                
                                                                                                  
 activation_94 (Activation)  (None, 74, 74, 32)           0         ['batch_normalization_94

In [24]:
model.compile(loss='binary_crossentropy', optimizer=tf.keras.optimizers.Adam(lr=0.00001), metrics=['accuracy']) # learing rate를 줄여 미세조정
model.fit(train_ds, validation_data='val_ds', epochs=5 )

Epoch 1/5


2023-06-01 12:34:31.403431: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


313/313 [==============================] - ETA: 0s - loss: 0.1194 - accuracy: 0.9683

IndexError: tuple index out of range